## Automated Machine Learning

### Datasets

We can choose one of the bellow listed datasets. Both of them are **binary classifications**.
- [Monkeypox Dataset](https://www.kaggle.com/datasets/muhammad4hmed/monkeypox-patients-dataset) (monkeypox.csv)
- [Heart Failure Dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction) (heart_failure.csv)

### Libraries
- [HyperOpt](https://hyperopt.github.io/hyperopt/)

### Instructions
1. Choose a dataset. Build and train a baseline for comparison. To construct the baseline you do the following:
    - Try a set of possible machine learning algorithms (13 algorithms) using their default hyperparamters and choose the one with the highest performance for comparison.

2. Based on the problem at hand, you study the potential pipeline structure,
algorithms or feature transformers at each step, hyper-parameters ranges. Use
hyperOpt with the potential search space to beat the baseline.

3. Monitor the the performance of you the constructed pipeline from the previous step across different time budgets (number of iterations) and report the least time budget that you are able to outperform the baseline.

4. Determine whether the difference in performance between the constructed pipeline and the baseline is statistically significant.

In [1]:
import numpy as np
import scipy as scp
import pandas as pd
import seaborn as sn

In [2]:
heart_df = pd.read_csv('./data/heart_failure.csv')
heart_df.sample(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
370,60,M,ASY,135,0,0,Normal,63,Y,0.5,Up,1
345,69,M,ASY,135,0,0,Normal,130,N,0.0,Flat,1
598,55,M,ASY,120,226,0,LVH,127,Y,1.7,Down,1


In [3]:
pox_df = pd.read_csv('./data/monkeypox.csv')
pox_df.sample(3)

,Patient_ID,Systemic Illness,Rectal Pain,Sore Throat,Penile Oedema,Oral Lesions,Solitary Lesion,Swollen Tonsils,HIV Infection,Sexually Transmitted Infection,MonkeyPox
16516,P16516,Fever,True,True,False,True,False,True,True,False,Positive
723,P723,None,False,False,True,True,True,True,True,True,Negative
6387,P6387,None,True,True,False,False,True,False,False,True,Positive
